#Fake News Classifier Using LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

###Loading Data From Kaggle

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c fake-news

 96% 9.00M/9.42M [00:00<00:00, 89.0MB/s]
100% 9.42M/9.42M [00:00<00:00, 87.1MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 42.7MB/s]
 97% 36.0M/37.0M [00:00<00:00, 65.2MB/s]
100% 37.0M/37.0M [00:00<00:00, 123MB/s] 


In [ ]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


###Importing Data 

In [ ]:
import pandas as pd
data=pd.read_csv("train.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


###Removing NaN Values from the dataset

In [ ]:
data=data.dropna()
data.reset_index(inplace=True)

###Extracting Dependent and Independent variables from data

In [ ]:
X = data.drop("label",axis=1)
y = data["label"]

###Checking The Shape of X and y

In [ ]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


###Importing the required Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

###Preprocessing The Text 



In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

### One Hot Encoding

In [ ]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 


###Applying Pad Sequences To Make Sentence Length Equal

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3964 4496  389]
 [   0    0    0 ... 1743 1290 4062]
 [   0    0    0 ... 3878 4206 4625]
 ...
 [   0    0    0 ... 3395 3893 4590]
 [   0    0    0 ... 2859 1735  474]
 [   0    0    0 ... 2096 2608 4972]]


#Creating Model for Train Set

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 34s 10ms/step - loss: 0.4809 - accuracy: 0.7530 - val_loss: 0.2033 - val_accuracy: 0.9118
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1386 - accuracy: 0.9447 - val_loss: 0.1992 - val_accuracy: 0.9198
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0889 - accuracy: 0.9684 - val_loss: 0.2197 - val_accuracy: 0.9122
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0680 - accuracy: 0.9746 - val_loss: 0.2589 - val_accuracy: 0.9059
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0517 - accuracy: 0.9821 - val_loss: 0.3219 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0357 - accuracy: 0.9883 - val_loss: 0.3412 - val_accuracy: 0.9065
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0256 - accuracy: 0.9916 - val_loss: 0.3783 - val_accuracy: 0.9075
Epoc

###Performance Metrics And Accuracy



In [ ]:
y_pred=model.predict_classes(X_test)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[3111,  308],
       [ 280, 2336]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9025683512841757